<a href="https://colab.research.google.com/github/SETIADEEPANSHU/ExtensiveVisionAIProgramme/blob/master/topic_model_with_pyldavis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import sys
# !{sys.executable} -m spacy download en
import re, numpy as np, pandas as pd
from pprint import pprint

# Gensim
import gensim, spacy, logging, warnings
import gensim.corpora as corpora
from gensim.utils import lemmatize, simple_preprocess
from gensim.models import CoherenceModel
import matplotlib.pyplot as plt

# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use', 'not', 'would', 'say', 'could', '_', 'be', 'know', 'good', 'go', 'get', 'do', 'done', 'try', 'many', 'some', 'nice', 'thank', 'think', 'see', 'rather', 'easy', 'easily', 'lot', 'lack', 'make', 'want', 'seem', 'run', 'need', 'even', 'right', 'line', 'even', 'also', 'may', 'take', 'come'])

%matplotlib inline
warnings.filterwarnings("ignore",category=DeprecationWarning)
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

In [0]:
import nltk
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [0]:
#uploading the dataset on to colab
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving one.csv to one (1).csv
User uploaded file "one.csv" with length 4684437 bytes


In [0]:
df = pd.read_csv('one.csv')
df = df.head(20000)
df.dropna()

,Unnamed: 0,Title,Body,Link
0,0.0,These “Amazon’s Choice” Sellers Are Bribing Cu...,Some sellers offer gift cards or freebies to c...,https://www.buzzfeednews.com/article/nicolengu...
1,1.0,Millions Of Americans Could Lose Power For Day...,The utility company did not definitively say h...,https://www.buzzfeednews.com/article/clarissaj...
2,2.0,Two Key Players In The Ukraine Controversy Spe...,Bank records reveal a whirlwind of spending at...,https://www.buzzfeednews.com/article/mikesalla...
3,3.0,Angry Parents Are Demanding Refunds For YouTub...,"The popular kids entertainer, previously invol...",https://www.buzzfeednews.com/article/katienoto...
4,4.0,Joe Biden Said Donald Trump “Should Be Impeached”,"""He's shooting holes in the Constitution,” Bid...",https://www.buzzfeednews.com/article/henrygome...
5,5.0,A Gunman Apparently Motivated By Anti-Semitism...,"Two people were killed in Halle, Germany, by a...",https://www.buzzfeednews.com/article/ryanhates...
6,6.0,"As Google Backlashes Have Risen And Fallen, So...",Google’s troubles have been moving from Europe...,https://www.buzzfeednews.com/article/alexkantr...
7,7.0,"Collin Martin, An Openly Gay Pro Soccer Player...",“I know that growing up as a closeted kid I co...,https://www.buzzfeednews.com/article/ryancbroo...
8,8.0,Jameela Jamil Explained Her George W. Bush Twe...,The star of The Good Place has been in a messy...,https://www.buzzfeednews.com/article/davidmack...
9,9.0,A YouTuber Has Been Accused By Fans Of A Scam ...,Alissa Violet's fans are now trolling her over...,https://www.buzzfeednews.com/article/tanyachen...


In [0]:
df['Title'] = df['Title'].str.replace("[^a-zA-Z#]", " ")

In [0]:
def sent_to_words(sentences):
    for sent in sentences:
        sent = re.sub('\S*@\S*\s?', '', sent)  # remove emails
        sent = re.sub('\s+', ' ', sent)  # remove newline chars
        sent = re.sub("\'", "", sent)  # remove single quotes
        sent = gensim.utils.simple_preprocess(str(sent), deacc=True) 
        yield(sent)  
# Convert to list
data = df.Title.values.tolist()
data_words = (sent_to_words(data))
#data_words1 = list(data_words)
data_words

<generator object sent_to_words at 0x7fa0ff611888>

In [0]:
ls

'one (1).csv'   one.csv   sample_data/


In [0]:
df.to_json(r'one.json')

In [0]:
ls

'one (1).csv'   one.csv   one.json   sample_data/


In [0]:
!pip install pyLDAvis

     |████████████████████████████████| 1.6MB 10.1MB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97711 sha256=e556ae471df94d21d391638c17adc478610231b35a484e39cab9d0d2420e03d5
  Stored in directory: /root/.cache/pip/wheels/98/71/24/513a99e58bb6b8465bae4d2d5e9dba8f0bef8179e3051ac414
Successfully built pyLDAvis


In [0]:
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

In [0]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [0]:
#titlesgroups = df(remove=('headers', 'footers', 'quotes'))
docs_raw = df.Title
print(len(docs_raw))

1090


In [0]:
tf_vectorizer = CountVectorizer(strip_accents = 'unicode',
                                stop_words = 'english',
                                lowercase = True,
                                token_pattern = r'\b[a-zA-Z]{3,}\b',
                                max_df = 0.5, 
                                min_df = 10)
dtm_tf = tf_vectorizer.fit_transform(df['Title'].values.astype('U'))   #docs_raw)
print(dtm_tf.shape)
#x = v.fit_transform(df['Review'].values.astype('U'))

(1090, 127)


In [0]:
tfidf_vectorizer = TfidfVectorizer(**tf_vectorizer.get_params())
dtm_tfidf = tfidf_vectorizer.fit_transform(df['Title'].values.astype('U')) 
print(dtm_tfidf.shape)

(1090, 127)


/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:1616: UserWarning: Only (<class 'numpy.float64'>, <class 'numpy.float32'>, <class 'numpy.float16'>) 'dtype' should be used. <class 'numpy.int64'> 'dtype' will be converted to np.float64.
  UserWarning)


In [0]:
# for TF DTM
lda_tf = LatentDirichletAllocation(n_components=5, random_state=0)    #n_components = can depend
lda_tf.fit(dtm_tf)
# for TFIDF DTM
lda_tfidf = LatentDirichletAllocation(n_components=5, random_state=0)
lda_tfidf.fit(dtm_tfidf)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=5, n_jobs=None,
                          perp_tol=0.1, random_state=0, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [0]:
pyLDAvis.sklearn.prepare(lda_tf, dtm_tf, tf_vectorizer)

/usr/local/lib/python3.6/dist-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.255400 -0.019974       1        1  22.263994
1     -0.096813 -0.110009       2        1  21.534019
0     -0.244729 -0.164121       3        1  20.261636
3      0.218156 -0.019672       4        1  19.245356
4     -0.132014  0.313775       5        1  16.694995, topic_info=    Category       Freq         Term      Total  loglift  logprob
110  Default  70.000000        trump  70.000000  30.0000  30.0000
78   Default  38.000000          old  38.000000  29.0000  29.0000
94   Default  72.000000       reveal  72.000000  28.0000  28.0000
92   Default  37.000000         quiz  37.000000  27.0000  27.0000
59   Default  39.000000         just  39.000000  26.0000  26.0000
125  Default  31.000000         year  31.000000  25.0000  25.0000
45   Default  35.000000        guess  35.000000  24.0000  24.0000
8    Default  68.000000        based  68.000000  23.0000  23.0000
79   Default  64.000000       people  64.000000  22.0000  22.0000
67   Default  27.000000         make  27.000000  21.0000  21.0000
62   Default  56.000000         know  56.000000  20.0000  20.0000
80   Default  24.000000  personality  24.000000  19.0000  19.0000
37   Default  27.000000         food  27.000000  18.0000  18.0000
17   Default  31.000000    character  31.000000  17.0000  17.0000
9    Default  24.000000         best  24.000000  16.0000  16.0000
36   Default  24.000000     favorite  24.000000  15.0000  15.0000
40   Default  21.000000         girl  21.000000  14.0000  14.0000
74   Default  19.000000       murder  19.000000  13.0000  13.0000
51   Default  22.000000         hong  22.000000  12.0000  12.0000
63   Default  22.000000         kong  22.000000  11.0000  11.0000
113  Default  22.000000      ukraine  22.000000  10.0000  10.0000
4    Default  19.000000       amazon  19.000000   9.0000   9.0000
65   Default  30.000000         like  30.000000   8.0000   8.0000
105  Default  33.000000         tell  33.000000   7.0000   7.0000
120  Default  21.000000      wedding  21.000000   6.0000   6.0000
106  Default  32.000000       things  32.000000   5.0000   5.0000
1    Default  18.000000     actually  18.000000   4.0000   4.0000
47   Default  20.000000    halloween  20.000000   3.0000   3.0000
118  Default  15.000000          way  15.000000   2.0000   2.0000
85   Default  15.000000       police  15.000000   1.0000   1.0000
..       ...        ...          ...        ...      ...      ...
74    Topic5  18.977893       murder  19.724284   1.7515  -3.0305
118   Topic5  14.805650          way  15.553045   1.7408  -3.2788
85    Topic5  14.802658       police  15.552604   1.7406  -3.2790
117   Topic5  13.759075        watch  14.509955   1.7369  -3.3521
103   Topic5  12.720996        suing  13.467602   1.7330  -3.4306
19    Topic5  12.720350      charged  13.467526   1.7330  -3.4306
87    Topic5  11.678899        prize  12.424886   1.7281  -3.5160
77    Topic5  11.678899        nobel  12.424886   1.7281  -3.5160
97    Topic5  11.678899   scientists  12.424886   1.7281  -3.5160
43    Topic5  10.633874          got  11.381838   1.7221  -3.6098
47    Topic5  16.145663    halloween  20.415517   1.5554  -3.1922
61    Topic5  13.577687       killed  17.196378   1.5538  -3.3654
48    Topic5   9.610555        harry  12.238806   1.5483  -3.7109
72    Topic5   9.611462        movie  13.187892   1.4737  -3.7109
71    Topic5   9.349776          men  13.017678   1.4591  -3.7385
126   Topic5  11.560136        years  17.830739   1.3567  -3.5262
60    Topic5   7.527235          key  11.870827   1.3345  -3.9553
122   Topic5  13.791180        woman  21.938476   1.3258  -3.3498
123   Topic5  17.882964          won  32.424225   1.1950  -3.0900
31    Topic5  11.312499          eat  20.928413   1.1749  -3.5479
73    Topic5   8.537978       movies  15.932655   1.1662  -3.8293
30    Topic5   6.460211          don  12.899545   1.0985  -4.1081
28    Topic5   

In [0]:
first_topic = lda_tf.components_[0]

In [0]:
top_topic_words = first_topic.argsort()[-10:]
top_topic_words 

array([ 35, 124,   1, 113,  51,  63,  65,  17,  79, 110])

In [0]:
for i in top_topic_words:
    print(tf_vectorizer.get_feature_names()[i])

fans
world
actually
ukraine
hong
kong
like
character
people
trump


In [0]:
for i,topic in enumerate(lda_tf.components_):
    print(f'Top 10 words for topic #{i}:')
    print([tf_vectorizer.get_feature_names()[i] for i in topic.argsort()[-10:]])
    print('\n')

Top 10 words for topic #0:
['fans', 'world', 'actually', 'ukraine', 'hong', 'kong', 'like', 'character', 'people', 'trump']


Top 10 words for topic #1:
['asylum', 'week', 'days', 'said', 'trump', 'case', 'people', 'wedding', 'things', 'know']


Top 10 words for topic #2:
['preferences', 'season', 'favorite', 'best', 'tell', 'food', 'guess', 'reveal', 'just', 'based']


Top 10 words for topic #3:
['probably', 'know', 'products', 'day', 'amazon', 'girl', 'personality', 'make', 'reveal', 'quiz']


Top 10 words for topic #4:
['killed', 'watch', 'woman', 'police', 'way', 'halloween', 'won', 'murder', 'year', 'old']




In [0]:
# 1. Wordcloud of Top N words in each topic
from matplotlib import pyplot as plt
from wordcloud import WordCloud, STOPWORDS
import matplotlib.colors as mcolors


In [0]:
cloud = WordCloud(stopwords=stop_words,
                  background_color='white',
                  width=2500,
                  height=1800,
                  max_words=10,
                  colormap='tab10',
                  color_func=lambda *args, **kwargs: cols[i],
                  prefer_horizontal=1.0)

topics = lda_tf.topics(formatted=False)

fig, axes = plt.subplots(2, 2, figsize=(10,10), sharex=True, sharey=True)

for i, ax in enumerate(axes.flatten()):
    fig.add_subplot(ax)
    topic_words = dict(topics[i][1])
    cloud.generate_from_frequencies(topic_words, max_font_size=300)
    plt.gca().imshow(cloud)
    plt.gca().set_title('Topic ' + str(i), fontdict=dict(size=16))
    plt.gca().axis('off')


plt.subplots_adjust(wspace=0, hspace=0)
plt.axis('off')
plt.margins(x=0, y=0)
plt.tight_layout()
plt.show()

AttributeError: ignored

AttributeError: ignored